In [ ]:
import pandas as pd
import numpy as np

files = [
    "Model1.csv", "Model2.csv", "Model3.csv",
    "Model4.csv", "Model5.csv", "Model6.csv", 
    "Model7.csv", "Model8.csv", "Model9.csv"
]

def get_final_submission(file_list):
    preds_prob = []
    
    for i, f in enumerate(file_list):
        p = pd.read_csv(f).iloc[:, 1].values
        if p.mean() > 1.0: p -= 1.0
        if i in [7, 8]: p = 1.0 - p
        preds_prob.append(np.clip(p, 0, 1))
        
    weights = [0.10, 0.05, 0.05, 0.25, 0.07, 0.05, 0.08, 0.20, 0.15]
    combined = np.zeros_like(preds_prob[0])
    
    for p, w in zip(preds_prob, weights):
        combined += p * w
        
    base_res = 1.0 + np.power(combined, 0.9)
    calib = np.load("calibration_params.npy")
    final_res = base_res + calib
    return final_res

final_output = get_final_submission(files)
sub = pd.read_csv("sample_submission.csv")
sub['voted'] = final_output
sub.to_csv("4주가대세_Final.csv", index=False)